In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
from math import isclose
from ipyexperiments import IPyExperimentsCPU
import ipyexperiments
import re, numpy as np

In [3]:
def consume_cpu_ram(n): return np.ones((n, n))

In [4]:
def check_defined(var_list, local_list): 
    for v in var_list: assert v in local_list, f"var {v} should exist in locals()"
        
def check_undefined(var_list, local_list):
    for v in var_list: assert v not in local_list, f"var {v} should not exist in locals()"

# --------------------------------------------------------------------- #
# the following functions work with the captured output
# output is captured by `%%capture output` from a cell before

# *** Experiment memory:
# RAM:  Consumed     Reclaimed
# CPU: 255.8 MB 255.8 MB ( 99.99%)
def get_consumed_reclaimed_size(output):
    pat = re.compile('Consumed\s+Reclaimed\nCPU:\s+([\d\.]+) MB\s+([\d\.]+) MB\s+\((\s*[\d\.]+)%\)', flags=re.MULTILINE)
    match = pat.findall(output)
    (consumed_size, reclaimed_size, reclaimed_pct) = map(float, match[0])
    return consumed_size, reclaimed_size, reclaimed_pct

def check_reclaimed(output):
    # basic checks
    to_match = [r'Experiment started', 'Experiment finished', r"x1, x2", r'Current state']
    for s in to_match: assert re.search(s, output), f"expecting string: {s}"

    # consumed/reclaimed checks
    consumed_size, reclaimed_size, reclaimed_pct = get_consumed_reclaimed_size(output)
    
    # compare: numbers are within 2% equal
    assert isclose(consumed_size, reclaimed_size, rel_tol=0.02), f"Reclaimed all memory: {consumed_size} == {reclaimed_size}"
    assert reclaimed_pct > 99.9, "99.9+% reclaimed"

def check_data(output, cpu_data):
    consumed_size, reclaimed_size, reclaimed_pct = get_consumed_reclaimed_size(output)

    # compare with data
    final_consumed_size_stats  = cpu_data.consumed /2**20
    final_reclaimed_size_stats = cpu_data.reclaimed/2**20
    # numbers are within 2% equal
    assert isclose(final_consumed_size_stats,  consumed_size, rel_tol=0.02), f"Consumed {final_consumed_size_stats} vs reclaimed {cpu.consumed} memory"
    assert isclose(final_reclaimed_size_stats, reclaimed_size, rel_tol=0.02), f"Reclaimed {final_reclaimed_size_stats} vs reclaimed {cpu.reclaimed} memory"

In [ ]:
"""test_version"""
assert ipyexperiments.__version__, "version check"

'test_version'

## Basic container test


In [ ]:
%%capture output
exp1 = IPyExperimentsCPU() # consume some cpu ram

x1 = consume_cpu_ram(2**12)
x2 = consume_cpu_ram(2**12)
check_defined(['x1', 'x2'], locals())

del exp1 # finish experiment
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print(output)
check_reclaimed(output)


*** Experiment started with the CPU-only backend

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 



*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU: 255.8 MB 255.8 MB (100.00%)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 





## Test stats data and selective vars preservation

In [ ]:
%%capture output
exp2 = IPyExperimentsCPU() 
x1 = consume_cpu_ram(2**12)

cpu_data = exp2.data
assert cpu_data
print(cpu_data)

exp2.keep_var_names('cpu_data')

x2 = consume_cpu_ram(2**12) 

cpu_data = exp2.data
print(cpu_data)

check_defined(['x1', 'x2', 'cpu_data'], locals())
    
cpu_data_final = exp2.finish() # finish experiment
print("\nNumerical data:\n", cpu_data_final)

check_defined(['cpu_data_final'], locals())
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print(output)
check_reclaimed(output)
check_data(output, cpu_data_final)


*** Experiment started with the CPU-only backend

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 


IPyExperimentMemory(consumed=134082560, reclaimed=0, available=16473374720)
IPyExperimentMemory(consumed=268439552, reclaimed=0, available=16338964480)

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Local variables:
Deleted: exp2, x1, x2
Kept:    cpu_data

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU: 256.0 MB 256.0 MB (100.00%)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 



Numerical data:
 IPyExperimentMemory(consumed=268439552, reclaimed=268427264, available=16606576640)



## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [ ]:
%%capture output
with IPyExperimentsCPU():
    x1 = consume_cpu_ram(2**12)
    x2 = consume_cpu_ram(2**12)
    
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print(output)
check_reclaimed(output)


*** Experiment started with the CPU-only backend

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 



*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU: 256.0 MB 256.0 MB (100.00%)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 





In [ ]:
%%capture output
with IPyExperimentsCPU() as exp: 
    x1 = consume_cpu_ram(2**12)
    z = "some data"
    x2 = consume_cpu_ram(2**12)
    exp.keep_var_names('z')
print(z)
check_defined(['z'], locals())
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print(output)
check_reclaimed(output)


*** Experiment started with the CPU-only backend

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.6 MB   15.5 GB  30.8 GB   0.43% 



*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Local variables:
Deleted: exp, x1, x2
Kept:    z

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU: 256.0 MB 256.0 MB ( 99.99%)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.7 MB   15.5 GB  30.8 GB   0.43% 


some data



## Deleting previously defined variables

Here we test that we **cannot** correctly detect and delete variables used in the scope of the experiment, but which were already defined prior to the experiment.

In [ ]:
%%capture output
x3 = 10
x4 = 20
with IPyExperimentsCPU(): 
    x1 = consume_cpu_ram(2**12)
    x2 = consume_cpu_ram(2**12)
    x3 = 10
    x4 = 10000
# x3 is the same value, so we can't tell the difference whether it was created before the experiment or not
# x4 is different from previous value, but it could have been modified by some function indirectly and not used directly in the experiment
check_defined(['x3', 'x4'], locals())
# x3 is newly defined, so it gets deleted
check_undefined(['x1', 'x2'], locals())

In [ ]:
output = str(output)
print(output)
check_reclaimed(output)


*** Experiment started with the CPU-only backend

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.7 MB   15.5 GB  30.8 GB   0.43% 



*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU: 256.0 MB 256.0 MB ( 99.99%)

*** Current state:
RAM:   Used      Free     Total    Util
CPU:  67.7 MB   15.5 GB  30.8 GB   0.43% 





In [ ]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()